# 1. don’t use pre-trained model

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
#To determine if your system supports CUDA
print("==> Check devices..")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Current device: ",device)

#Also can print your current GPU id, and the number of GPUs you can use.
print("Our selected device: ", torch.cuda.current_device())
print(torch.cuda.device_count(), " GPUs is available")

==> Check devices..
Current device:  cuda
Our selected device:  0
1  GPUs is available


In [3]:
print('==> Preparing dataset..')

==> Preparing dataset..


In [4]:
"""1.1"""
# The output of torchvision datasets are PILImage images of range [0, 1]
# We transform them to Tensor type
# And normalize the data
# Be sure you do same normalization for your train and test data

#The transform function for train data
transform_train = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
])

#The transform function for valid data
transform_valid = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
])

#The transform function for test data
transform_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
])

In [5]:
"""1.2""" 

#Use API to load Food-11 train dataset
trainset = torchvision.datasets.ImageFolder(root='/tmp/work/Food-11_03/training', transform=transform_train)

#Use API to load Food-11 valid dataset
validset = torchvision.datasets.ImageFolder(root='/tmp/work/Food-11_03/validation', transform=transform_valid)


#Use API to load Food-11 test dataset
testset = torchvision.datasets.ImageFolder(root='/tmp/work/Food-11_03/evaluation', transform=transform_test)

#The "download" option must be False in this Lab
#The cifar10 dataset we will provide in this path: /tmp/dataset-nctu
#Dataset definition need to know your customized transform function

In [6]:
"""1.3"""

#Create DataLoader to draw samples from the dataset
#In this case, we define a DataLoader to random sample our dataset. 
#For single sampling, we take one batch of data. Each batch consists 4 images
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=32,
shuffle=False, num_workers=2)

In [7]:
#Because Food-11 number the data classes in range [0,10]
#However, number representation is unreadable for humans
#So, we manually set the name of each class
classes = ('bread', 'dairy products', 'dessert', 'egg',
'fried food', 'meat', 'noodles & pasta', 'rice', 'seafood', 'soup', 'vegetables & fruits')

n_class=11

In [8]:
print('==> Building model..')

==> Building model..


In [9]:
import torch.nn as nn


net = torchvision.models.resnet18(pretrained=False)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, n_class)       
        


In [10]:
net = net.to(device) 

########################################################################

Define a Loss function and optimize 

########################################################################

In [11]:
print('==> Defining loss function and optimize..')

==> Defining loss function and optimize..


In [12]:
def lr_scheduler(optimizer, epoch, init_lr=0.001, lr_decay_epoch=7):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.1**(epoch // lr_decay_epoch))

    if epoch % lr_decay_epoch == 0:
        print('LR is set to {}'.format(lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return optimizer

In [13]:
import torch.optim as optim

#loss function
criterion = nn.CrossEntropyLoss()
#optimization algorithm
optimizer = optim.Adam(net.parameters(), lr=0.01)

# Decay LR by a factor of 0.1 every 30 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

########################################################################

Train the network Before training the model, we need to analysis the tensor variable.
Each variable have many attibute, like: .grad_fn, .require_grad, .data, .grad...etc. The ".grad_fn" attribute of "torch.Tensor" is an entry point into the function that has create this "torch.Tensor" variables. Because of ".grad_fn" flag, we can easily create a computing graph in the form of DAG(directed acyclic graph).

And then, the ".require_grad" attribute allows us to determine whether the backward propagation function is going to calculate the gradient of this "torch.Tensor" variable. If one variable has a false value of require_grad, it represent that you don't want to calculate this variable's gradient, and also its gradient will not be updated.

########################################################################

In [14]:
print('==> Training model..')

==> Training model..


In [15]:
#Set the model in training mode
#because some function like: dropout, batchnorm...etc, will have 
#different behaviors in training/evaluation mode
#[document]: https://pytorch.org/docs/stable/nn.html#torch.nn.Module.train

import time

since = time.time()

net.train()

for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    correct = 0
    exp_lr_scheduler.step()
    for i, (inputs, labels) in enumerate(trainloader, 0):
        
        #change the type into cuda tensor 
        inputs = inputs.to(device) 
        labels = labels.to(device) 

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # select the class with highest probability
        _, pred = outputs.max(1)
        # if the model predicts the same results as the true
        # label, then the correct counter will plus 1
        correct += pred.eq(labels).sum().item()
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f training accuracy: %.4f' %
                  (epoch + 1, i + 1, running_loss / 50, 100.*correct/len(trainset)))
            running_loss = 0.0
    #print('%d epoch, training accuracy: %.4f' % (epoch+1, 100.*correct/len(trainset)))
print('Finished Training')

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

[1,    50] loss: 2.903 training accuracy: 2.2096
[1,   100] loss: 2.314 training accuracy: 5.4227
[1,   150] loss: 2.236 training accuracy: 8.6053
[1,   200] loss: 2.235 training accuracy: 11.5042
[1,   250] loss: 2.146 training accuracy: 15.0213
[1,   300] loss: 2.099 training accuracy: 18.7411
[2,    50] loss: 2.073 training accuracy: 3.8820
[2,   100] loss: 2.083 training accuracy: 7.9769
[2,   150] loss: 2.064 training accuracy: 12.2643
[2,   200] loss: 2.024 training accuracy: 16.9471
[2,   250] loss: 2.010 training accuracy: 21.4069
[2,   300] loss: 2.024 training accuracy: 26.2112
[3,    50] loss: 1.981 training accuracy: 4.7233
[3,   100] loss: 1.992 training accuracy: 9.6797
[3,   150] loss: 1.997 training accuracy: 14.4131
[3,   200] loss: 2.000 training accuracy: 19.3189
[3,   250] loss: 2.000 training accuracy: 23.8597
[3,   300] loss: 1.968 training accuracy: 29.1303
[4,    50] loss: 1.900 training accuracy: 5.3923
[4,   100] loss: 1.971 training accuracy: 10.5413
[4,   15

[29,   100] loss: 1.134 training accuracy: 20.0892
[29,   150] loss: 1.168 training accuracy: 29.8196
[29,   200] loss: 1.121 training accuracy: 40.0061
[29,   250] loss: 1.151 training accuracy: 49.7466
[29,   300] loss: 1.142 training accuracy: 59.8115
[30,    50] loss: 1.120 training accuracy: 10.2270
[30,   100] loss: 1.117 training accuracy: 20.4237
[30,   150] loss: 1.146 training accuracy: 30.4075
[30,   200] loss: 1.125 training accuracy: 40.4723
[30,   250] loss: 1.128 training accuracy: 50.3852
[30,   300] loss: 1.115 training accuracy: 60.5210
[31,    50] loss: 1.022 training accuracy: 10.5717
[31,   100] loss: 1.008 training accuracy: 21.1839
[31,   150] loss: 0.974 training accuracy: 32.0191
[31,   200] loss: 0.970 training accuracy: 42.9657
[31,   250] loss: 0.962 training accuracy: 53.8516
[31,   300] loss: 0.963 training accuracy: 64.7780
[32,    50] loss: 0.996 training accuracy: 11.0278
[32,   100] loss: 0.919 training accuracy: 22.1366
[32,   150] loss: 0.979 trainin

[56,    50] loss: 0.756 training accuracy: 12.1123
[56,   100] loss: 0.782 training accuracy: 24.0624
[56,   150] loss: 0.770 training accuracy: 36.0835
[56,   200] loss: 0.783 training accuracy: 47.9424
[56,   250] loss: 0.755 training accuracy: 60.1460
[56,   300] loss: 0.752 training accuracy: 72.2887
[57,    50] loss: 0.738 training accuracy: 12.3150
[57,   100] loss: 0.734 training accuracy: 24.6199
[57,   150] loss: 0.720 training accuracy: 36.9248
[57,   200] loss: 0.764 training accuracy: 49.0878
[57,   250] loss: 0.785 training accuracy: 61.1494
[57,   300] loss: 0.759 training accuracy: 73.2516
[58,    50] loss: 0.731 training accuracy: 12.1326
[58,   100] loss: 0.775 training accuracy: 24.0928
[58,   150] loss: 0.744 training accuracy: 36.1747
[58,   200] loss: 0.736 training accuracy: 48.4594
[58,   250] loss: 0.753 training accuracy: 60.6730
[58,   300] loss: 0.778 training accuracy: 72.6130
[59,    50] loss: 0.770 training accuracy: 12.0616
[59,   100] loss: 0.817 trainin

[82,   300] loss: 0.665 training accuracy: 74.1537
[83,    50] loss: 0.711 training accuracy: 12.4468
[83,   100] loss: 0.705 training accuracy: 25.0456
[83,   150] loss: 0.686 training accuracy: 37.5633
[83,   200] loss: 0.700 training accuracy: 49.9290
[83,   250] loss: 0.670 training accuracy: 62.4265
[83,   300] loss: 0.702 training accuracy: 74.8226
[84,    50] loss: 0.711 training accuracy: 12.3860
[84,   100] loss: 0.660 training accuracy: 25.1267
[84,   150] loss: 0.707 training accuracy: 37.6343
[84,   200] loss: 0.716 training accuracy: 49.9797
[84,   250] loss: 0.706 training accuracy: 62.3758
[84,   300] loss: 0.686 training accuracy: 74.7111
[85,    50] loss: 0.699 training accuracy: 12.3758
[85,   100] loss: 0.684 training accuracy: 25.0152
[85,   150] loss: 0.708 training accuracy: 37.6242
[85,   200] loss: 0.685 training accuracy: 49.8378
[85,   250] loss: 0.635 training accuracy: 62.4975
[85,   300] loss: 0.680 training accuracy: 75.0557
[86,    50] loss: 0.722 trainin

In [17]:
#After training , save the model first
#You can saves only the model parameters or entire model
#Some difference between the two is that entire model 
#not only include parameters but also record hwo each 
#layer is connected(forward method).
#[document]: https://pytorch.org/docs/master/notes/serialization.html

print('==> Saving model..')

#only save model parameters
torch.save(net.state_dict(), './checkpoint_Lab3_2_1.t7')
#you also can store some log information
state = {
    'net': net.state_dict(),
    'acc': 100.*correct/len(trainset),
    'epoch': 100
}
torch.save(state, './checkpoint_Lab3_2_1.t7')

#save entire model
torch.save(net, './model_Lab3_2_1.pt')

print('Finished Saving')

==> Saving model..
Finished Saving


In [18]:
#Before testing, we can load the saved model
#Depend on how you save your model, need 
#different way to use it

print('==> Loading model..')

#If you just save the model parameters, you
#need to redefine the model architecture, and
#load the parameters into your model
net = torchvision.models.resnet18(pretrained=False)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, n_class)       
        
checkpoint = torch.load('./checkpoint_Lab3_2_1.t7')
net.load_state_dict(checkpoint['net'])

#If you save the entire model
net = torch.load('./model_Lab3_2_1.pt')
print(net)
print('Finished Loading')

==> Loading model..
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplac

In [19]:
print('==> Testing model..')

#Set the model in evaluation mode
#[document]: https://pytorch.org/docs/stable/nn.html#torch.nn.Module.eval 
net.eval()

==> Testing model..


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [20]:
correct = 0
total = 0
class_correct = list(0. for i in range(11))
class_total = list(0. for i in range(11))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device) 
        labels = labels.to(device) 
        outputs = net(images)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1



print('Total accuracy is: %.1f %% and loss is: %.2f' % (
    100 * correct / total,loss))
for i in range(11):
    print('Accuracy of %5s : %.1f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Total accuracy is: 76.8 % and loss is: 0.28
Accuracy of bread : 70.8 %
Accuracy of dairy products : 45.0 %
Accuracy of dessert : 92.9 %
Accuracy of   egg : 71.7 %
Accuracy of fried food : 61.4 %
Accuracy of  meat : 69.4 %
Accuracy of noodles & pasta : 82.7 %
Accuracy of  rice : 90.0 %
Accuracy of seefood : 83.3 %
Accuracy of  soup : 83.3 %
Accuracy of vegetables & fruits : 93.8 %


# 2. use pre-trained model

In [21]:
import torch.nn as nn


net = torchvision.models.resnet18(pretrained=True)
for param in net.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, n_class)       
        
net = net.to(device)

In [22]:
import torch.optim as optim

#loss function
criterion = nn.CrossEntropyLoss()
#optimization algorithm
optimizer = optim.Adam(net.fc.parameters(), lr=0.01)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [23]:
print('==> Training model..')

==> Training model..


In [24]:
#Set the model in training mode
#because some function like: dropout, batchnorm...etc, will have 
#different behaviors in training/evaluation mode
#[document]: https://pytorch.org/docs/stable/nn.html#torch.nn.Module.train

import time

since = time.time()

net.train()

for epoch in range(75):  # loop over the dataset multiple times
    running_loss = 0.0
    correct = 0
    exp_lr_scheduler.step()
    for i, (inputs, labels) in enumerate(trainloader, 0):
        
        #change the type into cuda tensor 
        inputs = inputs.to(device) 
        labels = labels.to(device) 

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        # select the class with highest probability
        _, pred = outputs.max(1)
        # if the model predicts the same results as the true
        # label, then the correct counter will plus 1
        correct += pred.eq(labels).sum().item()
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f training accuracy: %.4f' %
                  (epoch + 1, i + 1, running_loss / 50, 100.*correct/len(trainset)))
            running_loss = 0.0
    #print('%d epoch, training accuracy: %.4f' % (epoch+1, 100.*correct/len(trainset)))
print('Finished Training')

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_elapsed // 60, time_elapsed % 60))

[1,    50] loss: 2.138 training accuracy: 6.6491
[1,   100] loss: 1.246 training accuracy: 16.0855
[1,   150] loss: 1.379 training accuracy: 25.5625
[1,   200] loss: 1.274 training accuracy: 35.3436
[1,   250] loss: 1.223 training accuracy: 45.3983
[1,   300] loss: 1.173 training accuracy: 55.7470
[2,    50] loss: 1.195 training accuracy: 10.4095
[2,   100] loss: 1.210 training accuracy: 20.6872
[2,   150] loss: 1.257 training accuracy: 31.0866
[2,   200] loss: 1.298 training accuracy: 41.5569
[2,   250] loss: 1.092 training accuracy: 52.5441
[2,   300] loss: 1.175 training accuracy: 63.1360
[3,    50] loss: 1.365 training accuracy: 10.2676
[3,   100] loss: 1.191 training accuracy: 21.2852
[3,   150] loss: 1.392 training accuracy: 31.5224
[3,   200] loss: 1.147 training accuracy: 42.5400
[3,   250] loss: 1.171 training accuracy: 53.4665
[3,   300] loss: 1.331 training accuracy: 63.8962
[4,    50] loss: 1.181 training accuracy: 10.8251
[4,   100] loss: 1.188 training accuracy: 21.7515
[

[28,    50] loss: 0.806 training accuracy: 12.1022
[28,   100] loss: 0.757 training accuracy: 24.2956
[28,   150] loss: 0.736 training accuracy: 36.6410
[28,   200] loss: 0.758 training accuracy: 48.8648
[28,   250] loss: 0.764 training accuracy: 61.1089
[28,   300] loss: 0.743 training accuracy: 73.3428
[29,    50] loss: 0.802 training accuracy: 12.2137
[29,   100] loss: 0.818 training accuracy: 24.4476
[29,   150] loss: 0.828 training accuracy: 36.5092
[29,   200] loss: 0.755 training accuracy: 48.8445
[29,   250] loss: 0.789 training accuracy: 60.5818
[29,   300] loss: 0.706 training accuracy: 72.9982
[30,    50] loss: 0.790 training accuracy: 12.1528
[30,   100] loss: 0.774 training accuracy: 24.3766
[30,   150] loss: 0.768 training accuracy: 36.5802
[30,   200] loss: 0.798 training accuracy: 48.8242
[30,   250] loss: 0.801 training accuracy: 60.8251
[30,   300] loss: 0.734 training accuracy: 73.0286
[31,    50] loss: 0.763 training accuracy: 12.3556
[31,   100] loss: 0.742 trainin

[54,   300] loss: 0.801 training accuracy: 73.3428
[55,    50] loss: 0.756 training accuracy: 12.1833
[55,   100] loss: 0.836 training accuracy: 24.1942
[55,   150] loss: 0.817 training accuracy: 35.9822
[55,   200] loss: 0.793 training accuracy: 48.2262
[55,   250] loss: 0.681 training accuracy: 60.7946
[55,   300] loss: 0.801 training accuracy: 72.6840
[56,    50] loss: 0.766 training accuracy: 12.1731
[56,   100] loss: 0.711 training accuracy: 24.8125
[56,   150] loss: 0.766 training accuracy: 36.9957
[56,   200] loss: 0.755 training accuracy: 49.4628
[56,   250] loss: 0.805 training accuracy: 61.3521
[56,   300] loss: 0.804 training accuracy: 73.4036
[57,    50] loss: 0.786 training accuracy: 12.3556
[57,   100] loss: 0.855 training accuracy: 24.2246
[57,   150] loss: 0.826 training accuracy: 36.3268
[57,   200] loss: 0.760 training accuracy: 48.5506
[57,   250] loss: 0.762 training accuracy: 60.6933
[57,   300] loss: 0.810 training accuracy: 72.6941
[58,    50] loss: 0.837 trainin

In [25]:
#After training , save the model first
#You can saves only the model parameters or entire model
#Some difference between the two is that entire model 
#not only include parameters but also record hwo each 
#layer is connected(forward method).
#[document]: https://pytorch.org/docs/master/notes/serialization.html

print('==> Saving model..')

#only save model parameters
torch.save(net.state_dict(), './checkpoint_Lab3_2_2.t7')
#you also can store some log information
state = {
    'net': net.state_dict(),
    'acc': 100.*correct/len(trainset),
    'epoch': 100
}
torch.save(state, './checkpoint_Lab3_2_2.t7')

#save entire model
torch.save(net, './model_Lab3_2_2.pt')

print('Finished Saving')

==> Saving model..
Finished Saving


In [26]:
#Before testing, we can load the saved model
#Depend on how you save your model, need 
#different way to use it

print('==> Loading model..')

#If you just save the model parameters, you
#need to redefine the model architecture, and
#load the parameters into your model
net = torchvision.models.resnet18(pretrained=False)
num_ftrs = net.fc.in_features
net.fc = nn.Linear(num_ftrs, n_class)      
checkpoint = torch.load('./checkpoint_Lab3_2_2.t7')
net.load_state_dict(checkpoint['net'])

#If you save the entire model
net = torch.load('./model_Lab3_2_2.pt')
print(net)
print('Finished Loading')

==> Loading model..
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplac

In [27]:
print('==> Testing model..')

#Set the model in evaluation mode
#[document]: https://pytorch.org/docs/stable/nn.html#torch.nn.Module.eval 
net.eval()

==> Testing model..


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [28]:
correct = 0
total = 0
class_correct = list(0. for i in range(11))
class_total = list(0. for i in range(11))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device) 
        labels = labels.to(device) 
        outputs = net(images)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1



print('Total accuracy is: %.1f %% and loss is: %.2f' % (
    100 * correct / total,loss))
for i in range(11):
    print('Accuracy of %5s : %.1f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Total accuracy is: 81.3 % and loss is: 0.36
Accuracy of bread : 68.8 %
Accuracy of dairy products : 80.0 %
Accuracy of dessert : 85.7 %
Accuracy of   egg : 76.7 %
Accuracy of fried food : 65.9 %
Accuracy of  meat : 77.8 %
Accuracy of noodles & pasta : 92.3 %
Accuracy of  rice : 90.0 %
Accuracy of seefood : 100.0 %
Accuracy of  soup : 86.1 %
Accuracy of vegetables & fruits : 96.9 %
